# 基本资料

In [2]:
Cb = 0.85
Lpp = 215.6
Lpp_parent = 218
D = 19.31

# 最小干舷计算

## 夏季最小干舷

In [3]:
# 基本干舷
f0A = 2749+(2758-2749)*(215.6-215) # 参考规范p182，插值获得
f0B = 3505 + (3520-3505)*(215.6-215) # 参考规范p185，插值获得
f0 = f0B - (f0B-f0A)*0.6
f0 = int(f0)
print(f'''
基本干舷f0为{f0} mm
''')


基本干舷f0为3058 mm



In [4]:
# 船长大于100m，f1为0
f1 = 0

In [5]:
# 方形系数修正
f2 = (f0+f1)*((Cb+0.68)/1.36-1)
f2 = int(f2)
print(f'方形系数修正值f2为{f2} mm')

方形系数修正值f2为382 mm


In [6]:
# 型深修正
f3 = (D-Lpp/15)*250
f3 = int(f3)
print(f'型深修正值f3为{f3} mm')

型深修正值f3为1234 mm


In [7]:
# 上层建筑
S1 = 27357*Lpp/Lpp_parent # 首楼
B1 = 30658/32260
S2 = 15240*Lpp/Lpp_parent # 桥楼
B2 = 21338/32260
# 上层建筑高于标准值，因此没有造成长度缩放
E1 = S1*B1
E2 = S2*B2
E = (E1+E2)/1000
E_L = E/Lpp
E_L

0.16549907375282255

In [8]:
E1,E2

(25712.256399210546, 9969.343901897995)

In [9]:
# 插值
d = 7 + (14-7)/0.1*(E_L-0.1)
d

11.584935162697578

In [10]:
f4 = -1070*d/100
f4 = int(f4)
print(f'f4为{f4} mm')

f4为-123 mm


In [11]:
# 舷弧修正
# SA
D_parent = 19.6
# 尾垂线
Lbp_real = 415*D/D_parent
Lbp16_real = 0
Lbp13_real = 0
Lmid_real = 0
# 首垂线
Lfp_real = 2196*D/D_parent
Lfp16_real = 617*D/D_parent
Lfp13_real = 0
Lfp_real,Lfp16_real,Lbp16_real

(2163.5081632653055, 607.8709183673468, 0)

In [12]:
Lbp_std = 2046.67
Lbp16_std = 908.72
Lbp13_std = 229.23
Lmid_std = 0
Lfp_std = 4093.33
Lfp16_std = 2*Lbp16_std
Lfp13_std = 2*Lbp13_std

In [13]:
SA = 1*(Lbp_real - Lbp_std) + 3*(Lbp16_real-Lbp16_std) + 3*(Lbp13_real-Lbp13_std) + 1*(Lmid_real-Lmid_std)
SA /= 8
SA = round(SA,2)
SF = 1*(Lfp_real - Lfp_std) + 3*(Lfp16_real-Lfp16_std) + 3*(Lfp13_real-Lfp13_std) + 1*(Lmid_real-Lmid_std)
SF /= 8
SF = round(SF,2)
SA,SF

(-631.46, -866.74)

In [14]:
E1

25712.256399210546

In [15]:
# 首垂线处上层建筑高度
Hfp = 3316*D/D_parent
y = Hfp - 2300
L_ = E1/1000 # mm -> m
Sf = y*L_/3/Lpp
Sf = round(Sf,2)
Sp = 0
Sf,Sp

(38.44, 0)

In [16]:
S =Sf+Sp
S

38.44

In [17]:
l = (E1+E2)/1000
l

35.68160030110854

In [19]:
f5 = -((SF+SA)/2+S)*(0.75-l/2/Lpp)
f5 = round(f5,2)
f5

474.19

In [20]:
f0,f1,f2,f3,f4,f5

(3058, 0, 382, 1234, -123, 474.19)

In [25]:
# 夏季最小干舷
Fmin = f0 + f1 + f2 + f3 + f4 + f5
Fmin

5025.19

In [26]:
# 实际干舷
D = 19.31
d = 14.24
(D - d)*1000,(D-d)*1000 > Fmin

(5069.999999999998, True)

## 最小船首高度

In [27]:
from scipy.integrate import simpson
# 前体水线面面积系数，参考母船
Lpp_parent = 218
B_parent = 32.26
# 辛普森法，以下单位均为mm
last_height = 4249 # 最后一块儿的高度，这是由于最后一块儿和前一块儿的间隔不是定的delta,这里通过梯形法额外补上
last_delta = 224 
delta = 1392
H_list = [16054,15983,15878,15742,15568,15352,15051,14735,14389,13951,13459,12901,12283,11603,10849,9991,8889,7883,6505,4707]
A1 = simpson(H_list,dx=delta)
A2 = (H_list[-1] + last_height)*last_delta/2 # 最后一块儿的面积
A3 = 82433*16130 # 前半平行中体半面积
A = (A1 + A2+A3)*2/1e6
Cwf = A/(Lpp_parent/2*B_parent)
Cwf = round(Cwf,3)
Cwf

0.948

In [28]:
Fb = (6075*(Lpp/100) - 1875*(Lpp/100)**2 + 200*(Lpp/100)**3) * (2.08 + 0.609*Cb - 1.603*Cwf - 0.0129*(Lpp/(0.85*D)))
Fb = int(Fb) + 1
print(f'最小船首高度为{Fb} mm')

最小船首高度为5803 mm


In [29]:
# 自首垂线夏季最小干舷水线至露天甲板上边垂直距离
H1 = 2169 * D/D_parent # 首垂线处，水线至舷弧距离
L = Fmin + H1
L > Fb

True

In [30]:
H1

2136.907653061224

## 储备浮力

测投影面积应不小于  
$$
\frac{L}{1000}\{0.15[f_0(\frac{C_B+0.68}{1.36}-1)+f_3]\}+4(\frac{L}{3}+10)
$$

In [42]:
f0,f2,f3

(3058, 382, 1234)

In [43]:
Amin = Lpp/1000*0.15*(f0*((Cb+0.68)/1.36-1)+f3)+4*(Lpp/3+10)
Amin = round(Amin,2)
Amin

379.74

In [44]:
A1 = 0.15*Lpp*Fmin/1000
H2_list = [712,769,828,890,957,1030,1110,1198,1297,1406,1525,1656,1790,1954,2100,2200]
dx = 2180
A2 = simpson(H2_list,dx=dx)/1e6
H3_list = [2170,2169,2166,2163,2160,2156,2199,2213,2213,2214,2252,2238,2245,2231,2276,2269]
dx = 818
A3 = simpson(H3_list,dx=dx)/1e6
A = A1+A2+A3
A,A>Amin

(1304.2329019333333, True)